#### Install required packages

In [ ]:
%pip install -r requirements.txt

Load all environment variables (needed)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

Set the logging of all HTTP call

In [25]:
import logging
import httpx

logging.basicConfig(level=logging.DEBUG)
httpx_logger = logging.getLogger("httpx")
httpx_logger.setLevel(logging.DEBUG)

def set_logging(level: int):
    httpx_logger.setLevel(level)

### Without APIM calling directly OpenAI from Foundry

Now we will authenticate with the user credentials, this mean the user needs to proper RBAC to the instance of Azure OpenAI.  We won't pass using API

In [2]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

endpoint = os.getenv('AZURE_AI_FOUNDRY_ENDPOINT')
chat_completion_model_name = os.getenv('AZURE_CHAT_COMPLETION_MODEL')
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"

In [3]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)


Create helper methods

In [4]:
def ask_question(client: AzureOpenAI, question: str, model: str) -> str:

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": question,
            }
        ],
        max_completion_tokens=800,
        temperature=1.0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model=model
    )

    return response.choices[0].message.content

In [ ]:
answer = ask_question(client, "What I can do in Paris that is fun ?", os.getenv('AZURE_CHAT_COMPLETION_MODEL'))

print(answer)

#### Using OpenAI Endpoint

In the previous example the client is using Azure AI Foundry endpoint that look like **https://<youservicename>.cognitiveservices.azure.com/**

In [26]:
openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

# Close the previous client
client.close()

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=openai_endpoint,
    azure_ad_token_provider=token_provider,
)

In [ ]:
answer = ask_question(client, "What I can do in Paris that is fun ?", os.getenv('AZURE_CHAT_COMPLETION_MODEL'))

print(answer)

Now let's do it using the APIM

In [74]:
load_dotenv(override=True)

apim_openai_endpoint=os.getenv('APIM_OPENAI_ENDPOINT')
apim_subscription_key=os.getenv('APIM_SUBSCRIPTION_KEY')

print(apim_openai_endpoint)

https://genapimhg.azure-api.net/foundry


In [ ]:
token = DefaultAzureCredential().get_token("https://cognitiveservices.azure.com/.default")

In [ ]:
print(token.token)

In [ ]:
client.close()

#print(apim_subscription_key)

token = DefaultAzureCredential().get_token("https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=apim_openai_endpoint,   
    # default_headers={
    #     "api-key": apim_subscription_key
    # },
    #azure_ad_token=get_bearer_token_provider
    api_key=apim_subscription_key # Here we pass the APIM key   
)

In [ ]:
answer = ask_question(client, "What I can do in Rome that is fun ?", os.getenv('AZURE_CHAT_COMPLETION_MODEL'))

print(answer)

### Call the manual configured endpoint

In [19]:
load_dotenv(override=True)

manual_endpoint = os.getenv('MANUAL_APIM_ENDPOINT')
apim_openai_endpoint=os.getenv('APIM_OPENAI_ENDPOINT')
apim_subscription_key=os.getenv('APIM_SUBSCRIPTION_KEY')
print(apim_subscription_key)
print(manual_endpoint)

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=manual_endpoint,   
    api_key=apim_subscription_key
)

80cb9daca2da48adad31d93306a240c5
https://genapimhg.azure-api.net/test/


In [ ]:
answer = ask_question(client, "What I can do in Mexico that is fun ?", os.getenv('AZURE_CHAT_COMPLETION_MODEL'))

print(answer)

In [24]:
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=apim_openai_endpoint,   
    # default_headers={
    #     "api-key": apim_subscription_key
    # },
    azure_ad_token=get_bearer_token_provider
    #api_key=apim_subscription_key # Here we pass the APIM key   
)